# Pierwszy checkpoint 1. Projektu z przedmiotu WUM #

## Alicja Charuza, Mateusz Gałęziewski #

Zbiorem danych, na jakim pracujemy przy naszym projekcie jest [Fake News Dataset](https://www.kaggle.com/datasets/mohammadaflahkhan/fake-news-dataset-combined-different-sources). Poniżej przedstawiamy wyniki naszej pracy przy preprocessingu.

Najpierw może zobaczmy, jak wyglądają nasze dane.

In [3]:
import pandas as pd

In [4]:
df1 = pd.read_csv('original_data.csv')
df1

,Unnamed: 0,title,text,Ground Label
0,0,Ann Coulter Make Believes She Has ‘Gay Friend...,"It s hard to believe, but Donald Trump does ha...",fake
1,1,Rating: Moody‘s verbessert Ausblick für Russla...,bankensektor Der russische Staat werde die Ban...,fake
2,2,CAN WE ADD DIRTY MONEY ‘LAUNDERING’ To The Oba...,A member of the House Intelligence Committee i...,fake
3,3,Republicans on Obamacare repeal: 'We're going ...,WASHINGTON (Reuters) - House of Representative...,true
4,4,"Trump, on possible DACA deal, says border wall...",WASHINGTON (Reuters) - U.S. President Donald T...,true
...,...,...,...,...
69040,69040,Burundi opposition platform boycotts new round...,NAIROBI (Reuters) - Burundi s main opposition ...,true
69041,69041,Hillary’s Message To Former Miss Universe Cal...,Miss Universe 1996 Alicia Machado is now an Am...,fake
69042,69042,Cop Crashes Car And Runs Away When More Cops A...,The Daily Sheeple – by Ryan Banister \r\nAn aw...,fake
69043,69043,Trump Stole An Idea From North Korean Propaga...,Jesus f*cking Christ our President* is a moron...,fake


Nasza ramka danych zawiera dwie kolumny cech **title** i **text** oraz kolumnę **Ground Label** informującą nas, czy jest to fake news, czy też nie.

Pierwszą rzeczą, jaką zrobiliśmy było usunięcie newsów w języku innym niż angielski. Z naszych obserwacji zbioru danych wynika, że były one niemal wszystkie fałszywe, dlatego zdecydowaliśmy się z nich zrezygnować.

In [7]:
from langdetect import detect

def det_lang(df):
    def detect_language(text):
        try:
            lan = detect(text)
        except:
            lan = 'unknown'
        return lan

    df['language'] = df['text'].apply(detect_language)
    return df

In [9]:
df_lang = det_lang(df1)
counts = df_lang.groupby(['language', 'Ground Label']).size()
print(counts)

language  Ground Label
af        fake                3
ar        fake               22
ca        fake                3
cy        fake                3
da        fake                2
de        fake              133
el        fake                4
en        fake            40538
          true            26881
es        fake              175
et        fake                1
fi        fake                5
fr        fake               53
hr        fake                3
hu        fake                1
id        fake                2
it        fake               11
lt        fake                1
nl        fake                4
no        fake                4
pl        fake                4
pt        fake               15
ro        fake                5
ru        fake              203
sl        fake                1
so        fake                8
sv        fake                1
sw        fake               14
tl        fake                2
tr        fake               10
unknown   fake   

Oprócz tego usunęliśmy wiersze, które miały puste wartości w obu kolumnach lub też miały mniej niż 30 znaków, gdyż czasami w kolumnie **text** znajdowały się na przykład linki.

Drugą rzeczą, jaką zrobiliśmy jest połączenie kolumn **title** i **text** w jedną, a następnie wyczyszczenie powstałego tekstu poprzez usunięcie:
- wielkich liter
- interpunkcji
- liczb
- pojedynczych znaków
- stopwordów (czyli wyrazów takich jak an, a, the, and itp)
- przedrostków i przyrostków (stemizacja)

In [20]:
import re
import string
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import word_tokenize
# funkcja odpowiedzialna za czyszczenie pojedynczego wiersza

def clean_text(text, punctuation_chars):
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', ''.join(punctuation_chars)))
    # remove digits
    text = text.translate(str.maketrans('', '', string.digits))
    # remove all single characters
    pattern = r'(^| ).( |$)'
    text = re.sub(pattern, ' ', text)
    # remove multiple spaces
    text = re.sub(' +', ' ', text)
    # remove stopwords
    text = delete_stopwords(text)
    # stemming
    text = stemming(text)
    return text

# funkcja odpowiedzialna za usuwanie stopwordów

def delete_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# funkcja odpowiedzialna za stemizację

def stemming(text):
    words = word_tokenize(text)
    porter = PorterStemmer()
    stem_words = [porter.stem(word) for word in words]
    return ' '.join(stem_words)

W efekcie uzyskaliśmy taką postać danych.

In [12]:
df2 = pd.read_csv('clean_data.csv')
df2 = df[['full_text', 'Ground Label']]
df2

,full_text,Ground Label
0,ann coulter make believ gay friend make racist...,fake
1,add dirti money launder obama billion iran ran...,fake
2,republican obamacar repeal go get done washing...,true
3,trump possibl daca deal say border wall would ...,true
4,trump administr forc peac jame pinkerton origi...,fake
...,...,...
66771,burundi opposit platform boycott new round pea...,true
66772,hillari messag former miss univers call miss p...,fake
66773,cop crash car run away cop arriv daili sheepl ...,fake
66774,trump stole idea north korean propaganda parod...,fake


Zobaczmy bliżej, jak wygląda przykładowy full_text w porównaniu do tego, co było na początku.

In [14]:
df1['text'][0]

"It s hard to believe, but Donald Trump does have a sizable amount of supporters who agree with his ideas on immigration and guns. Unsurprisingly, one of those people is Ann Coulter. A racist in her own regard, she falls perfectly into Trump s fan base.After the shooting massacre at a gay nightclub in Orlando, Florida, Coulter tweeted out: To my gay friends: Please consider the possibility that Hillary s immigration policies might get you killed. See Trump s speech today. To my gay friends: Please consider the possibility that Hillary's immigration policies might get you killed. See Trump's speech today.  Ann Coulter (@AnnCoulter) June 13, 2016Okay, first, what gay person would be her friend? And before you get offended in any regard to that question (you know who you are) I m a lesbian and find Coulter repulsive on every level. Which leads to the bigger question  what person, in general, would be her friend?During his speech, Trump of course spent time doubling-down on his racist and 

In [15]:
df2['full_text'][0]

'ann coulter make believ gay friend make racist point tweet hard believ donald trump sizabl amount support agre idea immigr gun unsurprisingli one peopl ann coulter racist regard fall perfectli trump fan baseaft shoot massacr gay nightclub orlando florida coulter tweet gay friend pleas consid possibl hillari immigr polici might get kill see trump speech today gay friend pleas consid possibl hillari immigr polici might get kill see trump speech today ann coulter anncoult june okay first gay person would friend get offend regard question know lesbian find coulter repuls everi level lead bigger question person gener would frienddur speech trump cours spent time doublingdown racist islamophob remark muslim ban immigr unit statesdo coulter trump even recogn fact shooter whose name never mention want actual born unit state abl access arsen weaponri ammunit florida unit state lax gun law like almost everi shooter way truli combat sort issu tri prevent happen first placey alway chang mindset s

Następnie podzieliśmy zbiór danych na treningowe i testowe. Robimy to już po czyszczeniu tekstów, gdyż robienie tego razem nie wpływało na wynik.

In [21]:
from sklearn.model_selection import train_test_split
def split_data(df):
    x = df['full_text']
    y = df['Ground Label']

    # splitting data into train and test sets (and then splitting train test into train and test for us)
    x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                        random_state=123,
                                                        test_size=0.3,
                                                        shuffle=True)
    x_train_train, x_train_test, y_train_train, y_train_test = train_test_split(x_train, y_train,
                                                                                random_state=123,
                                                                                test_size=0.3,
                                                                                shuffle=True)
    return x_train_train, x_train_test, x_test, y_train_train, y_train_test, y_test

Na koniec przekształciliśmy naszą kolumnę z tekstem przy użyciu TF-IDF, czyli Term Frequency-Inverse Document Frequency. Ma to na celu przekształcenie naszych danych w dane numeryczne, abyśmy w kolejnym etapie byli w stanie przygotować model.

In [22]:
def tfidf(x_train_train, x_train_test, x_test):
    tfidfvectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
    tfidf_train_train = tfidfvectorizer.fit_transform(x_train_train)
    tfidf_train_test = tfidfvectorizer.transform(x_train_test)
    tfidf_test = tfidfvectorizer.transform(x_test)
    return tfidf_train_train, tfidf_train_test, tfidf_test

In [24]:
# Przykład dla zbioru danych treningowych
from scipy.sparse import load_npz

print(load_npz('train_data/x_train_train.npz'))

  (0, 28628)	0.035198335198265325
  (0, 108630)	0.05428777118207438
  (0, 166632)	0.022705542277684555
  (0, 60470)	0.03481588669784062
  (0, 85799)	0.032979096756262455
  (0, 160744)	0.0635480935883643
  (0, 57626)	0.03196559937771667
  (0, 135023)	0.06711695016675373
  (0, 119455)	0.03568232501669057
  (0, 45987)	0.07213574290869089
  (0, 140883)	0.034277694459140064
  (0, 30379)	0.03962275092864471
  (0, 157407)	0.08802594336151842
  (0, 8452)	0.02854628139148879
  (0, 150883)	0.030759466919202564
  (0, 146043)	0.056186449546128045
  (0, 70721)	0.04436949515098163
  (0, 133566)	0.04337403267948117
  (0, 145943)	0.060728083586618395
  (0, 23657)	0.05412518209399045
  (0, 35261)	0.035387715914827186
  (0, 94639)	0.025645401262050702
  (0, 168886)	0.19607660705230934
  (0, 80640)	0.19607660705230934
  (0, 48466)	0.04157334648006114
  :	:
  (32719, 88314)	0.03977518894866833
  (32719, 99187)	0.02558604868759703
  (32719, 18800)	0.03844388428135811
  (32719, 124016)	0.037357770258437215


Pierwsza wartość w krotce oznacza numer wiersza, a druga token_id, czyli numer słowa. Wartości wygenerowane przez funkcję mówią nam o tym, jak bardzo wpływa ono na treść tekstu, na podstawie częstości pojawiania się w tekście i we wszystkich tekstach.